# SQL 작성

In [1]:
import sys
import os
import pandas as pd
import xml.etree.ElementTree as ET
import cv2
from glob import glob
sys.path.append('../../src')
import DISData as DD
import multiprocessing as mp

## SQL 연결

In [2]:
doUT = DD.SQL()

host:  192.168.0.201
user:  diadmin
password:  ········


SQL connection done


## query 작성

In [3]:
query1 = (f'''CREATE TABLE IF NOT EXISTS AIhub_EyeMovement_IR_landmark_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(255) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(1000) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            label VARCHAR(250) NOT NULL,
            cam_loc VARCHAR(10) NOT NULL,
            landmark VARCHAR(1000));''')

In [4]:
query2 = (f'''INSERT INTO AIhub_EyeMovement_IR_landmark_info 
            (img_name, img_dir, img_format, img_width, img_height, 
            color_space, label_name, label_dir, label_format, label,
            cam_loc, landmark)
            VALUES (%s, %s, %s, %s, %s,  %s, %s, %s, %s, %s,  %s, %s)''')

In [5]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

Error during table creation: 1050: Table 'AIhub_EyeMovement_IR_landmark_info' already exists


In [6]:
value_list = []
error_list = []

label_path = 'Z:\\AIhub_EyeMovement\\data\\IR\\labeldata'
img_path = 'Z:\\AIhub_EyeMovement\\data\\IR\\rawdata'
label_lists = os.listdir(label_path)

for num, label_list in enumerate(label_lists):
    label_files = os.listdir(os.path.join(label_path,label_list))
    total_cnt = len(label_lists)
    color_space = 'IR'
    for cnt,label_file in enumerate(label_files):
        label_ = os.path.join(label_path,label_list,label_file)
        label_dir = label_
        label_name, label_format = os.path.splitext(os.path.basename(label_file))
        label_format = label_format.replace('.','')
        try:
            tree = ET.parse(label_)
        except:
            # print(cnt, label_file)
            continue
            
        xmls = tree.findall('.//image')

        for image in xmls:
            img_file = image.attrib['name'].split('/')[-1]
            cam_loc = img_file.split('_')[6]
            img_ = os.path.dirname(label_.replace('label','raw'))
            img_dir = os.path.join(img_,img_file)
            img_name, img_format = img_file.split('.')
            # print(img_dir)
            if os.path.exists(img_dir) and os.path.getsize(img_dir) != 0:
                img = cv2.imread(img_dir)
            else:
                # print(img_dir)
                error_list.append(img_dir)
                continue
            
            img_width = img.shape[0]
            img_height = img.shape[1]
            
            for polygon in image:
                landmarks = []
                landmark = []
                label = polygon.attrib['label']
                points = polygon.attrib['points'].split(';')
                
                for point in points:
                    x,y = point.split(',')
                    landmarks.append([x,y])
                    landmark = ",".join(map(str, [(int(float(x)), int(float(y))) for x,y in landmarks]))
                img_dir = img_dir.replace('Z:','DataBase')
                label_dir = label_.replace('Z:','DataBase')
                value1 = (img_name, img_dir, img_format, img_width, img_height, color_space,
                          label_name, label_dir, label_format,
                          label, cam_loc, landmark)
                value_list.append(value1)
                
            if len(value_list) >= 500:
                # print(value_list1[0])
                print(f'landmark {num}/{total_cnt}')
                doUT.insert_dataset_values(query2, value_list)
                value_list = []

doUT.insert_dataset_values(query2, value_list)
# print(value_list2)
value_list = []
print('done')      


landmark 0/217
insert 503 done
landmark 0/217
insert 504 done
landmark 0/217
insert 504 done
landmark 1/217
insert 504 done
landmark 1/217
insert 504 done
landmark 1/217
insert 504 done
landmark 2/217
insert 506 done
landmark 2/217
insert 502 done
landmark 2/217
insert 502 done
landmark 2/217
insert 500 done
landmark 3/217
insert 500 done
landmark 3/217
insert 500 done
landmark 3/217
insert 506 done
landmark 4/217
insert 506 done
landmark 4/217
insert 500 done
landmark 4/217
insert 500 done
landmark 5/217
insert 502 done
landmark 5/217
insert 506 done
landmark 5/217
insert 505 done
landmark 5/217
insert 504 done
landmark 6/217
insert 500 done
landmark 6/217
insert 507 done
landmark 6/217
insert 501 done
landmark 7/217
insert 505 done
landmark 7/217
insert 506 done
landmark 7/217
insert 506 done
landmark 8/217
insert 505 done
landmark 8/217
insert 501 done
landmark 8/217
insert 504 done
landmark 8/217
insert 504 done
landmark 9/217
insert 504 done
landmark 9/217
insert 504 done
landmark